In [25]:
import nltk
import numpy as np
from nltk.corpus import stopwords

In [26]:
with open('all_balanced.review','r',encoding='utf-8') as f:
    text = f.read()

In [27]:
bag_of_words = []

for line in text.split('\n'):
    seq = line.split('#label#:')
    if len(seq)==2:
        for key_value in seq[0].split(' '):
            bag_of_words.append(key_value.split(':')[0])
stopwords = stopwords.words('english')
bag_of_words = [w for w in bag_of_words if w.lower() not in stopwords] # Quitamos las stopwords
fdist = nltk.FreqDist(bag_of_words) # Sacamos las frecuencias
reduced_bow = [w for w in set(bag_of_words) if fdist[w]>50] # Separamos frecuencias que tengan mas de 50 repeticiones 
bigram_text = nltk.Text(bag_of_words) # Sacamos los biogramas
bigrams = list(nltk.bigrams(bigram_text))
top_bigrams = (nltk.FreqDist(bigrams)).most_common(250)

In [35]:
vectors = []
labels = []

for line in text.split('\n'):
    seq = line.split('#label#:')
    if len(seq)== 2:
        line_dict = {keyval.split(':')[0]:int(keyval.split(':')[1]) for keyval in seq[0].split(' ') if len(keyval.split(':'))==2}
        line_vector = []
        for word in reduced_bow:
            if word in line_dict.keys():
                
                line_vector.append(line_dict[word])
            else:
                line_vector.append(0)
        vectors.append(line_vector)
        labels.append(float(seq[1]))
        
        bigram_text = nltk.Text(line_dict.keys())
        bigrams = list(nltk.bigrams(bigram_text))
        for word in top_bigrams:
            if word in bigrams:
                line_vector.append(1)
            else:
                line_vector.append(0)
        vectors.append(line_vector)
        labels.append(float(seq[1]))
    
vectors = np.array(vectors)


In [41]:
from sklearn.model_selection import train_test_split

train_vectors, test_vectors, train_labels, test_labels = train_test_split(vectors, labels, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier().fit(train_vectors, train_labels)
print('Score for RandomForestClassifier = {}'.format(clf.score(test_vectors, test_labels)))

Score for RandomForestClassifier = 0.8845979100408905


In [52]:
type(labels)

list